In [134]:
import pyro
import numpy as np 
import torch
import pyro.distributions
import pyro.poutine as poutine
import copy
import time 
from tqdm import tqdm

In [75]:
### First abstract generative process from the perspective of an individual how performance results come into place
### One construction principle must be that a variable muts not be known for it to work
def abstractGP1(TI_mean_low,TI_mean_high,TI_var,TCadditive,TD_mean_low,TD_mean_high,TD_var,skill_mean,skill_var,luck_var,effort_var,p_low,p_high,group):
    # TC: Task Choice, TC = 1 (Free choice) , always observed by the person
    TC = pyro.sample('TC',pyro.distributions.Bernoulli(0.5)) 
    # TI: Task Importance, TI = 1 (High task importance), can be manipulated in the experiment
    TI_discrete = pyro.sample('TI_discrete',pyro.distributions.Bernoulli(0.5))
    TI_mean = TI_mean_high if TI_discrete else TI_mean_low
    # Free choice increases the task importance by TCadditive
    TI_mean = TI_mean + TCadditive * TC 
    # Sample task importance
    TI = pyro.sample('TI',pyro.distributions.Normal(TI_mean,TI_var))  
    # Sampling the skill level for the specific value
    skill = pyro.sample('skill',pyro.distributions.Normal(skill_mean,skill_var))
    # Sampling the effort level dependent on task importance 
    effort = pyro.sample('effort',pyro.distributions.Normal(TI,effort_var))
    # Luck as random gaussian noise
    luck = pyro.sample('luck', pyro.distributions.Normal(0,luck_var))  # maybe call it external, actualy specification can be different
    # TD: Task difficulty, TD_discrete = 1 (High Task Difficulty), can be manipulated in the experiment
    TD_discrete = pyro.sample('TD_discrete', pyro.distributions.Bernoulli(0.5))
    TD_mean = TD_mean_high if TD_discrete else TD_mean_low
    TD = pyro.sample('TD',pyro.distributions.Normal(TD_mean,TD_var))
    
    # Success prob is determined by 
    success_prob = p_low if (TD > (skill * effort + luck)) else p_high
    outcome = pyro.sample('outcome',pyro.distributions.Bernoulli(success_prob))
    
    # At the moment self_concept, self_worth, image all represent the same concept
    Self_concept = skill + effort 
    Self_worth = pyro.sample('self-worth',pyro.distributions.Normal(Self_concept,1))
    Image = group * (skill + effort) 
    return (outcome.item(),Self_worth,Image.item()) # results are torch tensors.


In [82]:
### Now for some settings we can sample and determine the outcome expectancy

# Setting: 
TI_mean_low = 5
TI_mean_high = 10
TI_var = 2
TD_mean_low = 5
TD_mean_high = 30
TD_var = 4
TCadditive = 2  
skill_mean = 5
skill_var = 2
luck_var = 2
effort_var = 3 
p_low = 0.01
p_high = 0.99
group = 0

# What is a guideline to pick those variables: Prior to any observations we want certain outcome expectations ?
# Maybe also dependent on variables such as self-esteem or self-concept.




In [83]:
# Simulator: 

n = 100 # number of runs

outcomes = []
positive_self_concept = []
negative_self_concept = []
for _ in range(n):
    results = abstractGP1(TI_mean_low,TI_mean_high,TI_var,TCadditive,TD_mean_low,TD_mean_high,TD_var,skill_mean,skill_var,luck_var,effort_var,p_low,p_high,group)
    outcomes.append(results[0])
    if results[0]:
        positive_self_concept.append(results[1])
    else:
        negative_self_concept.append(results[1])
    

## Print out the outcome expectancy and results on self_concept. If outcome is failure the value is counted for negative self_concept etc.

print('The success prob is {}'.format(sum(outcomes)/len(outcomes)))
print('A good self-concept is on average: {}'.format(np.mean(np.array(positive_self_concept))))
print('A negative self-concept is on average: {}'.format(np.mean(np.array(negative_self_concept))))


The success prob is 0.83
A good self-concept is on average: 15.579158782958984
A negative self-concept is on average: 9.562623023986816


In [78]:
## Now is the question: How do we use such a generative process: 

## As an example we take the teaching experiment
## Each experimental condition is given by a certain set of observed values

# Condition 1: No choice, high task importance, high task difficulty,failure: (Should show high SSB)
data_cond1 = {'TC':torch.tensor(0,dtype=torch.float32),'TI_discrete':torch.tensor(1,dtype=torch.float32),'TD_discrete':torch.tensor(0,dtype=torch.float32),'outcome':torch.tensor(0,dtype=torch.float32),'self-worth':15}
GP1_conditioned = pyro.condition(abstractGP1,data = data_cond1) # Condition the model: get back a callable

# Now what do we do with this conditioned model. First lets try to do LWinference on the latent variables.



## But how do we use this process for inference

internal_var = [] # variables that are unobserved and would imply internal attribution style
external_var = [] # variables that are unoberved and would imply external attribution style

# Perform counterfactual inference: Changing an unknown variable does it effect the outcome 
# If yes this variable is seen as a cause if not it is not
# Since we can only sample the outcome expectancy we need a cutoff value for the change to the conditioned model: 


# Perform hypthesis testing: Estimate the latent variables. 
# Question: How do we implement the biases ?


In [79]:
### Things to implement: 
# (1) In case the person has a negative self-esteem that we have a binary 0-1 variable that reduces the outcome expectancy

In [87]:
from collections import defaultdict
import numpy as np 

class LW():
    
    def __init__(self,model,observation,r = lambda x:x):
    
        '''
        model - function/generative process containing primitives (stochastic/deterministic)
        observation - dictionary with the variables observed - the observed values must be torch.tensors 
        r - function to apply to the latent variables (identity function)
        *args are the model parameters 
        **kwargs are the model keyword parameters
        '''
        
        self.model = model
        self.observation = observation 
        self.r = r
    
    def inferLW(self,L,*args,**kwargs):
        
        '''
        Input: L number of MC samples to get estimate
        Output: return estimate of the latent variables 
        (Side comment: If we have estimates for the parameters of a parameterized distribution we have a posterior from which we can sample)
        
        '''
        
        ### Condition the model on the observation 
        cond_model = pyro.condition(self.model,data = self.observation)
        logWs = []
        estimates = defaultdict(list)
        for _ in range(L):
            trace_DS = poutine.trace(cond_model).get_trace(*args,**kwargs)
            trace_DS.log_prob_sum()
            ### Get the observational nodes from the data structure
            obs_nodes = trace_DS.observation_nodes
            logWs.append(sum([trace_DS.nodes[elem]['log_prob_sum'] for elem in obs_nodes]))
            ### Add the samples of the latent variables 
            __ = [estimates[elem].append(trace_DS.nodes[elem]['value']) for elem in trace_DS.stochastic_nodes]
        ### Take the weighted average and apply r to the latent varibales 
        logWs = [np.exp(elem) for elem in logWs]
        logW_sum= sum(logWs)
        weights = np.array(logWs)/logW_sum.item() # can't divide by tensor
        for key in estimates.keys():
            estimates[key] = sum(self.r(np.array(estimates[key])) * weights)
        return estimates, logW_sum  

In [88]:
L = 10000
TI_mean_low = 5
TI_mean_high = 10
TI_var = 2
TD_mean_low = 5
TD_mean_high = 30
TD_var = 4
TCadditive = 2  
skill_mean = 5
skill_var = 2
luck_var = 2
effort_var = 3 
p_low = 0.01
p_high = 0.99
group = 0
model1_cond1 = LW(abstractGP1,data_cond1)
model1_cond1.inferLW(L,TI_mean_low,TI_mean_high,TI_var,TCadditive,TD_mean_low,TD_mean_high,TD_var,skill_mean,skill_var,luck_var,effort_var,p_low,p_high,group)

NameError: name 'abstractGP1' is not defined

In [89]:
### Concrete generative process - Experiment 1: Effect of Perceived Choice, expected outcome and actual outcome

### They judged personal responsibility as a variable, but that should be inference right?
### Wie krieg ich eine 
def concreteGP(TC_High_mean,TC_Low_mean,TC_var,skill_mean,skill_var,effort_var,luck_mean,luck_var,student_ability_mean,student_ability_var,PE_high_mean,PE_low_mean,PE_var,PO_var,Lower_Bound,Higher_Bound,self_worth_var,TC,PE):
    # The parameters of the function give the condition of the experiment
    # It would be nice if the abstract GP could take parameters such that we can adpot it to any experiment ? 
    # Rated 
    TC_mean = TC_High_mean if TC else TC_Low_mean
    PerceivedTaskChoice = pyro.sample('PerceivedTC',pyro.distributions.Normal(TC_mean,TC_var))
    skill = pyro.sample('skill_mean',pyro.distributions.Normal(skill_mean,skill_var)) 
    effort = pyro.sample('effort_mean',pyro.distributions.Normal(PerceivedTaskChoice,effort_var))
    luck = pyro.sample('luck_mean',pyro.distributions.Normal(luck_mean,luck_var))
    student_ability = pyro.sample('student_ability_mean',pyro.distributions.Normal(student_ability_mean,student_ability_var))
    PE_mean = PE_high_mean if PE else PE_low_mean
    PriorExperience = pyro.sample('PE_mean',pyro.distributions.Normal(PE_mean,PE_var))
    # Prior Experience means what is my expectation that it works well 
    # On a scale from 1-9, 1 is a failure and 9 is success
    summ = skill + effort + PriorExperience + luck + student_ability
    absolute_sum = abs(skill) + abs(effort) + abs(luck) + abs(student_ability) + abs(PriorExperience)
    PO_mean =  (summ/absolute_sum) * 10
    if PO_mean < Lower_bound:
        PO_mean = Lower_bound
    if PO_mean > Higher_bound:
        PO_mean = Higher_bound
    PerceivedOutcome = pyro.sample('PerceivedOutcome',pyro.distributions.Normal(PO_mean,PO_var))
    self_worth = pyro.sample('self-worth',pyro.distributions.Normal(skill + effort,self_worth_var))
    return PerceivedOutcome,self_worth
    
    
    
    

In [90]:
### Simulations to calibrate parameters: 
n = 100

# the outcome variable of the generative process is the outcome
TC = 0 
PE = 0

outcome = []

for _ in range(n):
    outcome.append(concreteGP(TC,PE))

print('Expected Outcome: {} '.format(np.mean(np.array(outcome))))
    

TypeError: concreteGP() missing 17 required positional arguments: 'TC_var', 'skill_mean', 'skill_var', 'effort_var', 'luck_mean', 'luck_var', 'student_ability_mean', 'student_ability_var', 'PE_high_mean', 'PE_low_mean', 'PE_var', 'PO_var', 'Lower_Bound', 'Higher_Bound', 'self_worth_var', 'TC', and 'PE'

In [91]:
### What do we want to do: Sample some perceived outcome depending on what feedback the person received: Failure or success feedback
### Try to find values for the external and internal variables that produce the perceived outcome 
### Value whether these variables are much different from the actual value if yes then we see it as a causal reason 

def expectation_based_inference_procedure(GP,internal,external,n,outcome,exp_param={},cond_param={},threshold=0,L=100):
    '''
    Param:
    (0) GP (function): a function which describes the generative process
    (1) internal (list): names of the variables that are internal
    (2) external (list): names of the variables that are external
    (3) exp_param: The GP should be parameterized such that the experiment is well described
    (4) cond_param: parameters such that one experimental condition is satisified
    (5) number of simulations to determine the expected outcome
    (6) actual observed outcome
    
    Output: Which variables can be seen as the causal variables
    '''
    # Für jede Person wollen wir folgendes machen: Wir haben ein expected outcome mit unserem generativen prozess und auch ein selbstwert
    # Wir haben ein wahrgenommenes outcome
    # Wenn die Differenz zu groß ist fragen wir nach den Ursachen
    # Dann konditionieren wir den Prozess auf das wahre outcome und den alten selbstwert (wollen wir erhalten)
    # Bestimmen mit LW die means der neuen variablen
    # Updaten den Prozess mit den neuen means
    # Das Maß jetzt ist um wie viel ich die means updaten musste. Somit bekomme ich für jeder Person eine Kausalitätsscore
    # Die kann ich dann in eine Skala von 1-9 umwandeln und mit den Resultaten vergleichen 
    # n ist dabei die Anzahl an Personen in einer Gruppe 
    
    causality = []
    count = 0
    for _ in range(n):
        print('{} %'.format(count/n))
        count +=1
        exp_outcome,self_worth = GP(*list(exp_param.values()),*list(cond_param.values()))
        print('expected-outcome: {}'.format(exp_outcome))
        actual_outcome = pyro.sample('actual_outcome',pyro.distributions.Normal(outcome,2))
        print('actual_outcome: {}'.format(actual_outcome))
        updated_model = LW(GP,{'PerceivedOutcome':actual_outcome,'self-worth':self_worth})
        ### Inference on the update model with LW-inference
        mean_values,log_p = updated_model.inferLW(L,*(list(exp_param.values()) + list(cond_param.values())))
        ### Need to extract the mean values of the sampled parameters
        internal_change = 0
        external_change = 0
        for key in mean_values.keys():
            if key in set(internal.keys()):
                internal_change += abs(mean_values[key]-exp_param[key])
            if key in set(external.keys()):
                external_change += abs(mean_values[key]-exp_param[key])
        causality_ind = ((internal_change)/(internal_change + external_change)) * 8 + 1 # change to a score on 1-9
        causality.append(causality_ind) 
    
    print('The causality score is: {}'.format(np.mean(np.array(causality))))
    return causality
    

In [92]:
### Setting the condition parameters and experimental parameters
TC_High_mean = 7 
TC_Low_mean = 3
TC_var = 1.5
skill_mean = 5
skill_var = 2
effort_var = 2
luck_mean = 0
luck_var = 2
student_ability_mean = 0
student_ability_var = 3
PE_high_mean = 7
PE_low_mean = 3
PE_var = 1.5
PO_var = 2
Lower_Bound = 3
Higher_Bound =7
self_worth_var = 2
expr_param = {'TC_High_mean':TC_High_mean,'TC_Low_mean':TC_Low_mean,'TC_var':TC_var,'skill_mean':skill_mean,'skill_var':skill_var,'effort_var':effort_var,'luck_mean':luck_mean,'luck_var':luck_var,'student_ability_mean':student_ability_mean,'student_ability_var':student_ability_var,'PE_high_mean':PE_high_mean,'PE_low_mean':PE_low_mean,'PE_var':PE_var,'PO_var':PO_var,'Lower_Bound':Lower_Bound,'Higher_Bound':Higher_Bound,'self_worth_var':self_worth_var}

####
TC = 0 # task choice
PE = 0 # prior expectation negative
## in these condition we should see a high outcome expectation
cond_param = {'TC':TC,'PE':PE}
### What do we set the variables of internal to ? 
internal = {'skill_mean': 9}
external = {'luck_mean': -2, 'student_ability_mean':-2}
n = 100
outcome = 8




In [93]:
### Maybe its better to break the inference up in subprocedures: 

causality_scores = expectation_based_inference_procedure(concreteGP,internal,external,n,outcome,expr_param,cond_param)



0.0 %


NameError: name 'Lower_bound' is not defined

In [94]:
import matplotlib.pyplot as plt
plt.hist(causality_scores)

NameError: name 'causality_scores' is not defined

In [ ]:
### Implementing hypothesis testing as a human inference method 

def hypothesis_inference(model,hypothesis,fixed_param,dist_param,cond_param,n):
    '''
    hypothesis is a dictionary which contains for each random variable a proposed parameter value which represents
    the hypothesis to be tested 
    '''
    ranking = [] # contains tuples: the first element is the name of the hypothesis and the second is the self-worth value
     
    for key,value in hypothesis:
        # Do inference on the model conditioned on the hypothesis
        updated_model = LW(model, {key:value})
        mean_values, log_P = updated_model.inferLW(L,*(list(exp_param.values()) + list(dist_param.values())+ list(cond_param.values())))
        ### Get the self-worth value for the model after setting the new parameters 
        ## we need to copy the dictionary
        updated_param = dist.copy()
        for key in mean_values.keys():
            updated_param[key] = mean_values[key]
            
            
        
        
    for elem in ranking:
        # test the hypothesis until one is confirmed
    




In [ ]:
a = {'a':3,'b':4}
b = a.copy()
b['a']=5
print(a,b)

In [95]:
### Question is whether I can design my GP in such a way that it simplifies inference

def concreteGP(TC_High_mean,TC_Low_mean,TC_var,skill_mean,skill_var,effort_var,luck_mean,luck_var,student_ability_mean,student_ability_var,PE_high_mean,PE_low_mean,PE_var,PO_var,Lower_Bound,Higher_Bound,self_worth_var,TC,PE):
    # The parameters of the function give the condition of the experiment
    # It would be nice if the abstract GP could take parameters such that we can adpot it to any experiment ? 
    # Rated 
    TC_mean = TC_High_mean if TC else TC_Low_mean
    PerceivedTaskChoice = pyro.sample('PerceivedTC',pyro.distributions.Normal(TC_mean,TC_var))
    skill = pyro.sample('skill_mean',pyro.distributions.Normal(skill_mean,skill_var)) 
    effort = pyro.sample('effort_mean',pyro.distributions.Normal(PerceivedTaskChoice,effort_var))
    luck = pyro.sample('luck_mean',pyro.distributions.Normal(luck_mean,luck_var))
    student_ability = pyro.sample('student_ability_mean',pyro.distributions.Normal(student_ability_mean,student_ability_var))
    PE_mean = PE_high_mean if PE else PE_low_mean
    PriorExperience = pyro.sample('PE_mean',pyro.distributions.Normal(PE_mean,PE_var))
    # Prior Experience means what is my expectation that it works well 
    # On a scale from 1-9, 1 is a failure and 9 is success
    summ = skill + effort + PriorExperience + luck + student_ability
    absolute_sum = abs(skill) + abs(effort) + abs(luck) + abs(student_ability) + abs(PriorExperience)
    PO_mean =  (summ/absolute_sum) * 10
    if PO_mean < Lower_bound:
        PO_mean = Lower_bound
    if PO_mean > Higher_bound:
        PO_mean = Higher_bound
    PerceivedOutcome = pyro.sample('PerceivedOutcome',pyro.distributions.Normal(PO_mean,PO_var))
    self_worth = pyro.sample('self-worth',pyro.distributions.Normal(skill + effort,self_worth_var))
    return PerceivedOutcome,self_worth

## 3 types of variables:
# variables that indicate a condition -> should be categorical 
# variables that indicate determine a parameter
# LW gibt mir für jedes sample statement einen wert -> ok aber dann sample ich nicht mehr sondern nehme den mittelwert
# Wollen wir nicht umbedingt 
# Die einzigen Verteilungen die wir parameterisieren können, sind die die keine parent nodes haben 


In [143]:
class Human():
    '''
    Base class for individual participants of an experiment
    Each participant is initialized with a generative process of the experiment which depends on the experimental settings 
    Each GP also has parameters of the experiment which can be tuned to produce numerical results close to the ones found in the experiment 
    Example given: In the experiment humans should rate for how important they consider the task on a 1-9 scale
    Then we have a param task_importance_mean, task_importance_var 
    These can then be tuned to the data after the experiment
    Assume we model it with a normal probability: We
    In case this is an experimental condition which was manipulated we should have 
    task_importance_mean_lower, task_importance_var_lower
    task_importance_mean_high, task_importance_var_high
    '''
    
    def __init__(self,GP,process_param,personality = [], situation = [],n = 1, obs = []):
        self.IT = GP
        self.personality = personality  # a list of all personal variables
        self.situation = situation # list of all the situational variables 
        self.process_param = process_param # list of the process parameters, need to be calibrated on the data
        self.n = n # number of participants of this type
        self.obs = obs # outcome observations for the participants
        
    def add_situation(new_aspect):
        assert isinstance(new_aspect,characteristic)
        self.situation.append(new_aspect)
        
    def add_personality(new_trait):
        assert isinstance(new_aspect,characteristic)
        self.situation.append(new_trait)
    
    def get_expectation(self):
        return self.IT(*self.personality,*self.situation,*self.process_param)
    
        
    def callibrate(data):
        pass
        
    def self_worth_protection_inference(self,L):
        '''
        Implement the self_worth_protection inference method with the new way of writing GP
        '''
        personal_causality = []

        for i in tqdm(range(self.n)):
            exp_outcome,self_worth = self.IT(*self.personality,*self.situation,*self.process_param)
            actual_outcome = self.obs[i]
            data = {'Expected Outcome':actual_outcome,'Self-Worth':self_worth}
            updated_model = LW(self.IT,data)
            ### Inference on the update model with LW-inference
            mean_values,log_p = updated_model.inferLW(L,*self.personality,*self.situation,*self.process_param)
            ### Need to extract the mean values of the sampled parameters
            internal_change = 0
            external_change = 0
            external_count = 0
            internal_count = 0
            for elem in self.personality:
                if elem.name not in data.keys():
                    internal_count += 1
                    internal_change += abs(mean_values[elem.name]-elem.get_param()[0])
            for elem in self.situation:
                if elem.name not in data.keys():
                    external_count += 1
                    external_change += abs(mean_values[elem.name]-elem.get_param()[0])
            external_change *= internal_count/external_count
            internal_change *= external_count/internal_count
            causality_ind = ((internal_change)/(internal_change + external_change)) * 8 + 1 # change to a score on 1-9
            personal_causality.append(causality_ind) 
        return personal_causality

    
    def hypothesis_testing_inference(self,L):
        '''
        General idea: each hypothesis is given by observing a certain value
        Under each hypothesis we have a certain self-worth 
        We rank all hypothesis regarding their self-worth 
        Then we test the hypothesis. How tough the test is depends on the self-worth
        High self-worth gives easy testing and vice versa 
        Easy testing means the hypothesis is accepted 
        As soon as we have accepted an hypothesis we stop the process and have the causal reason 
        Question now is how to generate these hypothesis - especially the parameter values  
        '''
        
        
    
        

In [144]:
### In what way do we implement GP 
### In psychology we try to measure ein Merkmal 
### ein Merkmal wird gemessen auf einer Skala 
### Wir müssen eine Verteilung annehmen (später vielleicht empirische Verteilung)
### ist es external(situation) oder internal(position)

class characteristic():
    '''
    Base class for characteristics 
    '''
    # class attribute: Dict of distributions which possibly represent the parameter
    distributions = {'Normal':pyro.distributions.Normal,'Bernoulli': pyro.distributions.Bernoulli,'Binomial':pyro.distributions.Binomial}
    
                     
    def __init__(self,name,scale,type_dist,type_characteristic,param = None):
        self.name = name 
        self.scale = scale # scale of the method which records the characteristic
        self.type_dist = type_dist # string such as 'Normal' 
        self.type_characteristic = 1 if type_characteristic=='internal' else 0 # type: internal (=1) or external (=0)
        self.param = param
    def init_dist(self):
        return self.distributions[self.type_dist](*self.param)
    
    def set_param(self,param):
        self.param = param # a list containing the parameters that describe the distirbution
    
    def get_param(self):
        return self.param
        
    

### What are good ways to represent scales -> maybe another class: domain knowledge needed what are typical scales 

class scale(object):
    
    def __init__(self,name,minn,maxx,bool_cont):
        self.name = name
        self.minn = minn
        self.maxx = maxx
        self.bool_cont = bool_cont
    
    
        


In [145]:
### Testing out the characteristic class 
### Problem: What to do with latent states for which we do not measure it on any scale ??
Likert1_9 = scale('Likert: 1-9',1,9,True)
Likert1_18 = scale('Likert: 1-18',1,18,True)


TI = characteristic('task importance',Likert1_9,'Normal','external')
mean = 2
var = 1
TI.set_param([mean,var])
Skill = characteristic('Skill',Likert1_9,'Normal','internal')
mean = 5 
var = 2
Skill.set_param([mean,var])
Effort = characteristic('Effort',Likert1_9,'Normal','internal')
mean = 5 
var = 2
Effort.set_param([mean,var])
### If two characteristics are added up, multiplied: try to combine their scales 
Self_Worth = characteristic('Self-Worth',Likert1_18,'Normal','internal')
Student_Ability = characteristic('Student Ability',Likert1_9,'Normal','external')
mean = 5
var = 2
Student_Ability.set_param([mean,var])
Prior_Experience = characteristic('Prior Experience',Likert1_9,'Normal','external')
mean = 5
var = 2
Prior_Experience.set_param([mean,var])
Expected_Outcome = characteristic('Expected Outcome',Likert1_9,'Normal','internal')
Expected_Outcome.set_param([0,2])

In [146]:
### Writing a GP with the characteristics classes 

def GP_versuch(Skill,Effort,Self_Worth,Prior_Experience,Student_Ability,TI,c_1,c_2):
    ti = pyro.sample(TI.name,TI.init_dist())
    skill = pyro.sample(Skill.name,Skill.init_dist()) #1-9 scale
    effort = pyro.sample(Effort.name,Effort.init_dist()) #1-9 scale
    prior_experience = pyro.sample(Prior_Experience.name,Prior_Experience.init_dist()) #1-9 scale
    student_ability = pyro.sample(Student_Ability.name,Student_Ability.init_dist()) #1-9 scale
    ## Determine Self-Worth 
    var = Skill.get_param()[1] + Effort.get_param()[1]
    Self_Worth.set_param([skill + effort, var])
    self_worth = pyro.sample(Self_Worth.name,Self_Worth.init_dist())
    self_worth = c_1 * ti
    ## Determine Expected Outcome
    outcome_mean = skill + effort + prior_experience - c2*student_ability
    Expected_Outcome.set_param([outcome_mean,Expected_Outcome.get_param()[1]])
    expected_outcome = pyro.sample(Expected_Outcome.name, Expected_Outcome.init_dist())
    return expected_outcome, self_worth



In [148]:
c1,c2 = 1,1
traits = [Skill,Effort,Self_Worth,Prior_Experience]
student_ability = [Student_Ability,TI]
### Observationen erzeugen 
mean = 2
var = 2
N = 100 
obs = []
for elem in range(N):
    obs.append(pyro.distributions.Normal(mean,var).rsample())
    
participant1 = Human(GP_versuch,process_param = [c1,c2],personality = traits ,situation = student_ability,n = N,obs = obs )
participant1.get_expectation()
L = 100
personal_causality = participant1.self_worth_protection_inference(L)
print(np.mean(np.array(personal_causality)))        

100%|██████████| 100/100 [00:14<00:00,  7.00it/s]

6.04938719231671


In [84]:
pyro.distributions.Normal(mean,var).rsample()

tensor(6.2476)

In [149]:
### Writing a GP for the new experiment 
### Extending the Human class with a new inference method 

### Since the actual scale does not matter for each trait we sample from a normal distribution centered around 0 
mu = 5 
sigma = 2 
normal_scale = scale('normal_support',-np.inf,np.inf,True)
Skill = characteristic('Skill','Normal')
Skill.set_param([mu,sigma])
External 

### Two new moderators: Improvement Possibilities, Self-awareness

def GP_versuch(Skill,Effort,Self_Worth,Prior_Experience,TI,c_1,c_2):
    skill = pyro.sample(Skill.name,Skill.init_dist()) 
    effort = pyro.sample(Effort.name,Effort.init_dist())
    
    var = Skill.get_param()[1] + Effort.get_param()[1]
    Self_Worth.set_param([skill + effort, var])
    self_worth = pyro.sample(Self_Worth.name,Self_Worth.init_dist())
    self_worth = c_1 * ti
    ## Determine Expected Outcome
    outcome_mean = skill + effort + prior_experience - TD
    Expected_Outcome.set_param([outcome_mean,Expected_Outcome.get_param()[1]])
    expected_outcome = pyro.sample(Expected_Outcome.name, Expected_Outcome.init_dist())
    return expected_outcome, self_worth

### How could we implement the inference method -> two competeing methods 
### Compare it to a self image -> compare parameters to a self image -> measure negative effect
### Normal inference system is given by just by conditioning on expected_outcome nothing else
### Comparing to self-image only if self-awarenss is high -> otherwise normal inference 
### Possibility of improvement: We need a measure how chagneable the traits are ? 
### If it is changeable






In [153]:
a = np.inf